In [17]:
#coding = utf-8
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
import pyes
import json
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

host = "140.118.175.209:9200"
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
es = Elasticsearch(['http://140.118.175.209:9200/'])

fid1 = "157941490891336" #ford
fid2 = "134004310003557" #pxmart
fid3= "139624776093433" #ikea
print es

<Elasticsearch([{u'host': '140.118.175.209', u'scheme': 'http', u'port': 9200}])>


In [51]:
def getLikes_Fans(fid):
    a = es.search(
        index = "facebook_nested",
        doc_type="fanpage",
         body={
                "query": {
                    "term": {
                        "fid": fid
                    }
                },
                 "size":200
        },
        size = 200

    )
    df = pd.read_json(json.dumps(a["hits"]["hits"]))
    q = df["_source"][0]
    
#     datetime.datetime.fromtimestamp(1463034913204/1000).strftime('%Y-%m-%d') ,
    return  (q["likes_count"], q["talking_about_count"])


likes, talking_about = getLikes_Fans(fid1)
print likes, talking_about

122421 7067


In [50]:
def getTotalPost(fid):
    b = es.count(
    index = "facebook_nested",
            doc_type="post",
             body={
                    "query": {
                        "term": {
                            "fid": fid
                        }
                    },
                     "size":200
            }
    )
    df = pd.read_json(json.dumps(b))
    return df["count"]["total"]

totalpost = getTotalPost(fid3)
print totalpost


1120


In [46]:
def boxPlotNumofPostPerMonth(fid,host,lowerbound,upperbound,pngFilePath):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq) 
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')

    data=[]
    for r in result.aggs["month"]["buckets"]:
        for pid in r["pid"]["buckets"]:
            monthyear=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%Y/%m')
            data.append([pid["doc_count"],monthyear,pid["key"],r["key"]])

    df=pd.DataFrame(data,columns={'date','count','fid','pid'})

    sns.set(style="ticks")
    sns_plot=sns.boxplot(x="date", y="count", data=df, palette="PRGn")
    fig=sns_plot.get_figure()
    plt.ylim(lower_bound, upper_bound)
    fig.savefig(pngFilePath)


lower_bound = 0
upper_bound = 600
fid=fid1
pngFilePath="output3-1.png"
boxPlotNumofPostPerMonth(fid,host,lower_bound,upper_bound,pngFilePath)

In [42]:
def boxPlotFBFansPageComp(fids,host,pngFilePath,outlierFilter):
    data=[]
    for fid in fids:
        conn = pyes.es.ES(host)
        tq = pyes.query.TermQuery(field="fid", value=fid)
        tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
        qsearch = pyes.query.Search(tq) 
        qsearch.agg.add(tagg) 
        result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
        #print json.dumps(result.aggs,indent=2) 
        for fidpid_summary in result.aggs["pid"]["buckets"]:
            #print fidpid_summary["key"] + ',' + str(fidpid_summary["doc_count"])
            if fidpid_summary["doc_count"] < outlierFilter or outlierFilter < 0:
                data.append([fidpid_summary["doc_count"],fid,fidpid_summary["key"]])

    df=pd.DataFrame(data,columns={'count','fid','fidpid'})
    sns.set(style="ticks")
    sns_plot=sns.boxplot(x="fid", y="count", data=df, palette="PRGn")
    fig=sns_plot.get_figure()
    fig.savefig(pngFilePath)
    
    
fids=[fid1,fid2,fid3]
pngFilePath="output.png"
outlierFilter=400
boxPlotFBFansPageComp(fids,host,pngFilePath,outlierFilter)